In [7]:
from torch_geometric.data import HeteroData
import torch
import pandas as pd
import numpy as np

In [11]:
users_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/users.csv")
products_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/all_products.csv")
user_product_edge_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/user_product_edges.csv")
also_bought_edges_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/also_bought_edges.csv")
also_viewed_edges_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/also_viewed_edges.csv")
bought_together_edges_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/bought_together_edges.csv")

In [28]:
user_product_edge_pd.head()

,overall,reviewerID,asin,date,helpfulness_ratio
0,5,A2XVJBSRI3SWDI,0000031887,2013-11-04,0.0
1,2,A2R3K1KX09QBYP,0000031887,2013-02-17,1.0
2,1,A19PBP93OF896,0000031887,2013-03-21,0.0
3,4,A1P0IHU93EF9ZK,0000031887,2014-01-23,0.0
4,5,A2G5TCU2WDFZ65,0000031887,2013-01-19,0.0


In [15]:
also_bought_edges_pd.head()

,asin,also_bought
0,0000031887,B003AVKOP2
1,0000031887,B00D103F8U
2,0000031887,B008F0SU0Y
3,0000031887,B003AVEU6G
4,0000031887,B002C3Y6WG


In [17]:
also_viewed_edges_pd.head()

,asin,also_viewed
0,0000037214,B00JO8II76
1,0000031887,B00538F5OK
2,0000031887,B003AVKOP2
3,0000031887,B008F0SU0Y
4,0000031887,B002GZPTRQ


In [18]:
bought_together_edges_pd.head()

,asin,bought_together
0,0123456479,B000BMTCK6
1,0123456479,B0006JCGUM
2,1608322130,B003USWPHA
3,245600258X,B004LK9GZ0
4,3293015344,B007VE0OI6


In [70]:
class AsinIdMap:
    def __init__(self, products_pd):
        self.products_pd = products_pd
        self.asin_to_idx = {}
        self.product_idx_to_asin = {}
        self._build()
    
    def _build(self):
        for idx, row in self.products_pd.iterrows():
            self.asin_to_idx[row['asin']] = idx
            self.product_idx_to_asin[idx] = row['asin']
    
    def get_idx(self, asin):
        return self.asin_to_idx[asin]
    
    def get_asin(self, product_idx):
        return self.product_idx_to_asin[product_idx]
    
    def get_count(self):
        return len(self.asin_to_idx)
    
def build_product_to_product_edge_index(edges_pd, asin_id_map, right_key):
    product_to_product_edge_index = []
    errors = []
    unique_products = {}
    for idx, row in edges_pd.iterrows():
        try:
            asin1 = row['asin']
            product1_idx = asin_id_map.get_idx(asin1)
        except:
            #errors.append(f"Cannot map {asin1} to product idx")
            continue
        
        try:
            asin2 = row[right_key]
            product2_idx = asin_id_map.get_idx(asin2)
        except:
            #errors.append(f"Cannot map {asin2} to product idx")
            continue
        
        unique_products[asin1] = 1
        unique_products[asin2] = 1
        
        product_to_product_edge_index.append([product1_idx, product2_idx])
        
    return torch.tensor(product_to_product_edge_index).t().contiguous(), unique_products, errors

In [45]:
asin_id_map = AsinIdMap(products_pd)

In [69]:
also_bought_edges_tensor, unique_products1, errors =  build_product_to_product_edge_index(also_bought_edges_pd, asin_id_map, 'also_bought')

In [73]:
#also_viewed_edges_tensor, unique_products2, errors =  build_product_to_product_edge_index(also_viewed_edges_pd, asin_id_map, 'also_viewed')

In [68]:
bought_together_edges_tensor, unique_products3, errors =  build_product_to_product_edge_index(bought_together_edges_pd, asin_id_map, 'bought_together')

In [71]:
unique_products = []
unique_products.extend(unique_products1)
#unique_products.extend(unique_products2)
unique_products.extend(unique_products3)
unique_products = list(set(unique_products))

In [3]:
data = HeteroData()

In [74]:
# Dimension for product features
product_feature_dim = 128
num_products = len(unique_products)

data['product'].x = torch.randn(num_products, product_feature_dim)

# Similar products
# data['product', 'also_viewed', 'product'].edge_index = also_viewed_edges_tensor

# Complementary products
data['product', 'bought_together', 'product'].edge_index = bought_together_edges_tensor
data['product', 'also_bought', 'product'].edge_index = also_bought_edges_tensor

data

HeteroData(
  product={ x=[588359, 128] },
  (product, also_viewed, product)={ edge_index=[2, 1000] },
  (product, bought_together, product)={ edge_index=[2, 554094] },
  (product, also_bought, product)={ edge_index=[2, 6279224] }
)